In [1]:
#installing/importing dependencies
from __future__ import print_function


!pip install gmaps
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

import gmaps
import pickle
import os.path
import numpy as np
import pandas as pd


from urllib.parse import urlencode
from ipywidgets import IntProgress
from urllib.request import urlopen
from IPython.display import display
from urllib.request import HTTPError
from urllib.request import urlretrieve
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

gmaps.configure(api_key=api_key)

Requirement already up-to-date: google-api-python-client in /anaconda3/lib/python3.7/site-packages (1.7.8)
Requirement already up-to-date: google-auth-httplib2 in /anaconda3/lib/python3.7/site-packages (0.0.3)
Requirement already up-to-date: google-auth-oauthlib in /anaconda3/lib/python3.7/site-packages (0.3.0)


In [5]:
#reading in credentials
with open('apikey.txt') as f:
    api_key = f.readline()
    f.close

with open('ID1.txt') as f:
    ID1 = f.readline()
    f.close
    
with open('ID2.txt') as f:
    ID2 = f.readline()
    f.close

In [7]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.

name1 = 'Sheet1!A2:A'

def sheet_to_list(ID, name):
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=ID,
                                range=name).execute()
    values = result.get('values', [])

    colleges = [loc[0] for loc in values if len(loc) != 0]
    return colleges

colleges1 = sheet_to_list(ID1, name1)
ranks1 = sheet_to_list(ID1, 'Sheet1!C2:C')
colleges2 = sheet_to_list(ID2, 'Sheet1!A2:A')
locs2 = sheet_to_list(ID2, 'Sheet1!B2:B')
applies = sheet_to_list(ID1, 'Sheet1!D2:D')

In [63]:
frame = pd.DataFrame({'Colleges':colleges1, 'applies':applies, 'ranks':ranks1})
filtered = frame[frame['applies'] == 'TRUE']
filtered_colleges = filtered['Colleges'].values
filtered_ranks = filtered['ranks'].values

In [8]:
cities_data = pd.read_csv('uscitiesv1.4.csv') #instead of doing this, query google!

In [17]:
def query_wikipedia(college):
    '''Determines which city a college is in based on its Wikipedia entry.
    Not the most rigorous, but it works (largely)!'''
    try: 
        query = f'''https://en.wikipedia.org/wiki/{college}'''
        urlpath = urlopen(query)
    except HTTPError:
        college = college.replace(" ", "_")
        query =f'''https://en.wikipedia.org/wiki/{college}'''
        urlpath = urlopen(query)
    string = urlpath.read().decode('utf-8')
    city_index = string.split('>').index('Location</th') + 4
    if ', ' not in string.split('>')[city_index]:
        city = string.split('>')[city_index][:-3]
        state_index = string.split('>').index('Location</th') + 8
        state = string.split('>')[state_index][:-3]
    else:
        both = string.split('>')[city_index].split(', ')
        city = both[0]
        state = both[1][:-3]
    try:
        abbrev = state_to_abbrev(state)
    except ValueError:
        try:
            state_index = string.split('>').index('Location</th') + 8
            state = string.split('>')[state_index][:-3]
            abbrev = state_to_abbrev(state)
        except ValueError:
            state_index = string.split('>').index('Location</th') + 7
            state = string.split('>')[state_index][:-5]
            abbrev = state_to_abbrev(state)
    return city, abbrev


In [23]:
#now query wikipedia for the state abbreviations.
def state_to_abbrev(state, verbose=False):
    '''Checks against the Wikipedia page for U.S. state abbreviations.'''
    query = 'https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations'
    urlpath = urlopen(query)
    string = urlpath.read().decode('utf-8')
    abbrev_index = string.split('>').index(state + '</a') + 12
    abbrev = string.split('>')[abbrev_index][:-6]
    return abbrev

In [72]:
def find_coords(colleges=None, cities_list=None, states_list=None, ranks_list=None):
    '''Takes in list of colleges. Returns DataFrame with city, state, 
    latitude, longitude, and rank of college.'''
    lat, lng = [], []
    cities, states = [], []
    ranks = []
    if type(colleges) == list:
        iterations = len(colleges)
    else:
        iterations = len(cities_list)
    f = IntProgress(min=0, max=iterations) # instantiate the progress bar
    display(f) # display the progress bar
    for i in range(iterations):
        f.value += 1
        if not cities_list:
            try:
                city, state = query_wikipedia(colleges[i])
            except IndexError:
                print('Need to at least input a list of colleges!')
        else:
            city, state = cities_list[i], states_list[i]
            if len(state) > 2:
                state = state_to_abbrev(state)
        corr_city = cities_data['city']==city #check the city from Wikipedia against the data.
        corr_state = cities_data['state_id']==state
        corr_df = cities_data[corr_city & corr_state]
        try:
            lat += [corr_df['lat'].values[0]]
            lng += [corr_df['lng'].values[0]]
            cities += [city]
            states += [state]
            if type(ranks) == list:
                ranks += [ranks_list[i]]
        except IndexError:
            print(f"Can't determine where where {city} is!")
    coords_frame = pd.DataFrame({'City':cities, 'State':states, 'Latitude':lat, 
                                 'Longitude':lng, 'Ranks':ranks})
    return coords_frame

In [53]:
cities2 = [string.split(', ')[0] for string in locs2]
states2 = [string.split(', ')[1] for string in locs2]
frame2 = find_coords(cities_list=cities2, states_list=states2)

IntProgress(value=0, max=19)

In [75]:
#make the ranking optional
frame1 = find_coords(colleges=list(filtered_colleges), ranks_list=list(filtered_ranks))

top_ranked_1 = frame1[frame1['Ranks'] == '1']
mid_ranked_1 = frame1[frame1['Ranks'] == '2']
low_ranked_1 = frame1[frame1['Ranks'] == '3']

plot_1_cities1 = top_ranked_1[['Latitude', 'Longitude']]
plot_1_cities2 = mid_ranked_1[['Latitude', 'Longitude']]
plot_1_cities3 = low_ranked_1[['Latitude', 'Longitude']]

plot_2_cities = frame2[['Latitude', 'Longitude']]

top_rank_1_layer = gmaps.symbol_layer(
    plot_1_cities1, fill_color='green', stroke_color='green', scale=4)
mid_ranked_1_layer = gmaps.symbol_layer(
    plot_1_cities2, fill_color='yellow', stroke_color='yellow', scale=4)
low_ranked_1_layer = gmaps.symbol_layer(
    plot_1_cities3, fill_color='red', stroke_color='red', scale=4)
all_2_layer = gmaps.symbol_layer(
    plot_2_cities, fill_color='purple', stroke_color='purple', scale=3)
fig = gmaps.figure()
fig.add_layer(top_rank_1_layer)
fig.add_layer(mid_ranked_1_layer)
fig.add_layer(low_ranked_1_layer)
fig.add_layer(all_2_layer)
fig

IntProgress(value=0, max=13)

Can't determine where where University Park is!


Figure(layout=FigureLayout(height='420px'))